# Catalog Wrangling Exercise  
*:auth: Nate Stevens (PNSN)*

In this exercise we'll use ObsPy and ObsPlus to create a highly translatable earthquake  
catalog of located events from our analyses and those in the USGS Comprehensive Catalog  
(ComCat) and do some quick intercomparisons between the two. Finally we'll save these  
catalogs into a variety of standardized formats (schema) that are easy to re-load and  
re-analyze with ObsPy and Pandas.

In [ ]:
import os
from pathlib import Path
from glob import glob
import pandas as pd
from obspy import read_events, UTCDateTime
import obsplus
from obspy.geodetics import gps2dist_azimuth, kilometer2degrees
from obspy.clients.fdsn import Client


In [ ]:
# TODO: Make sure this points at wherever you saved your HypoDD outputs
ROOT = Path.cwd()
DATA = ROOT/'data'
CATD = ROOT/'catalog_files'
os.makedirs(str(CATD), exist_ok=True)
print(f'The data directory is registered as {DATA}')

In [ ]:
# Load the HypoDD output into an ObsPy `Catalog` object
flist = glob(str(DATA/'*.pha'))
for _e, _f in enumerate(flist):
    if _e == 0:
        cat = read_events(_f)
    else:
        cat += read_events(_f)


display(cat)

In [ ]:
# Use ObsPlus to show a DataFrame representation of events (takes a little time)
df_events = cat.to_df()

### Compare this table the ORIGIN table in the ANSS schema

https://ncedc.org/db/Documents/NewSchemas/PI/v1.6.4/PI.1.6.4/Content/Tbl_388b5374f81611d6bcce00c04f794c81.htm

#### ...and look at all those empty fields, just waiting for you to populate them!



In [ ]:
# Display our new table (conveniently formatted in nearly ANSS EVENT table format!)
display(df_events)

## Althought the ObsPlus documentation is sometimes sparese on examples, their coding is quite good!
Let's turn all of our picks into a dataframe

In [ ]:
# Turns out the *.pha I/O for ObsPy has a little bug, so we need to apply a small correction to assign network and station codes to the correct fields
try:
    df_picks = cat.arrivals_to_df()
except:
    for event in cat.events:
        for pick in event.picks:
            sn = pick.waveform_id.station_code
            pick.waveform_id.station_code=sn.split('.')[0]
            pick.waveform_id.network_code=sn.split('.')[1]
    df_picks = cat.arrivals_to_df()

### Compare this to the ARRIVAL and ASSOCARO tables in the ANSS schema

#### ARRIVAL
https://ncedc.org/db/Documents/NewSchemas/PI/v1.6.4/PI.1.6.4/Content/Tbl_388b5400f81611d6bcce00c04f794c81.htm

#### ASSOCARO (Association of Arrivals and Origins)
https://ncedc.org/db/Documents/NewSchemas/PI/v1.6.4/PI.1.6.4/Content/Tbl_388b542ef81611d6bcce00c04f794c81.htm

In [ ]:
display(df_picks)

# Now that we've populated an ObsPy Catalog object, we can write into a bunch of different formats

QuakeML is a well-described, extensible schema for seismic event (meta)data exchange  
https://quake.ethz.ch/quakeml 

- ObsPy saves `Catalog` objects in this format as default  

**BUT** before you go saving everything as one big QuakeML file, be warned that they can get large and slow to read from disk.  

You can find one of my past sins against easily accessible (albiet well formatted) data here:  
https://zenodo.org/records/8393876


Instead, let's put our catalog into a tidy directory structure with a client interface!  
Another place where `ObsPlus` shines!  

In [ ]:
# Initialize an event bank
ebank = obsplus.EventBank(base_path=CATD/'EventBank',
                          path_structure='{year}/{month}/{day}/{hour}',
                          name_structure='{event_id_end}',
                          format='quakeml')

In [ ]:
# Add events to eventbank, and take a look at your file directory!
ebank.put_events(cat)

In [ ]:
# Get a summary of events in your event bank
display(ebank.read_index())

### Now let's prove to ourself that this EventBank thingy is persistent

In [ ]:
# DELETE the EventBank Object in our session
del ebank
try:
    display(ebank)
except NameError:
    print('ebank object does not exist, as expected')

In [ ]:
# Re-initialize connection to the EventBank
ebank = obsplus.EventBank(base_path=CATD/'EventBank')
display(ebank)
# Note that the `path_structure` or `name_structure` key-word arguments we defined are saved!
print('Our Event Bank values')
display(ebank.path_structure)
display(ebank.name_structure)
print('Default values')
print('{year}/{month}/{day}')
print('{time}_{event_id_short}')

In [ ]:
# Query a subset of events
# Read the index (a pandas DataFrame)
df_index = ebank.read_index()
# Subset by origin times
_df_index = df_index[(df_index.time >= pd.Timestamp('2022-12-20T20:00:00')) & (df_index.time <= pd.Timestamp('2022-12-20T21:40:00'))]
# Get events from your event bank
cat = ebank.get_events(event_id=_df_index.event_id.values)

display(cat)


### Let's modify some event metadata and submit it to our EventBank
In this case, let's add distances and back-azimuths to associated phases  

In [ ]:
# Let's populate some source-receiver geometry information
client = Client('IRIS')
nets = ','.join(list(df_picks.network.unique()))
stas = ','.join(list(df_picks.station.unique()))
inv = client.get_stations(network=nets, station=stas, level='channel',starttime=UTCDateTime('20221220'), endtime=UTCDateTime('20221221'))

# Use ObsPlus added methods to convert the inventory into a dataframe
df_stations = inv.to_df()

display(df_stations)

In [ ]:
# Add the maximum azimuthal gap to each origin
# Here's a starting point:

# Iterate across events
origin_gaps = []
for event in cat.events:
    # Iterate across origins
    for origin in event.origins:
        olon = origin.longitude
        olat = origin.latitude
        # Iterate across associated arrivals
        bazs = set([])
        for arrival in origin.arrivals:
            # Get pick observations
            pick = arrival.pick_id.get_referred_object()
            # Get station location
            network = pick.waveform_id.network_code
            station = pick.waveform_id.station_code
            _df_sta = df_stations[(df_stations.network==network) & (df_stations.station==station)][['station','network','latitude','longitude']]
            try:
                slon = _df_sta.longitude.values[0]
                slat = _df_sta.latitude.values[0]
            except:
                continue
            # Get distances
            dist_m, seaz, esaz = gps2dist_azimuth(slat, slon, olat, olon)
            # Convert distance to degrees
            arrival.distance = kilometer2degrees(dist_m*1e-3)
            # Assign back-azimuth
            arrival.azimuth = esaz

## A task for the HACK-A-THON, get azimuthal gaps into your EventBank

#             bazs.add(esaz)

        
#         # Calculate gaps
#         bazs = list(bazs)
#         bazs.sort()
#         gaps = [bazs[_e+1] - bazs[_e] for _e in range(len(bazs)-1)] + [360 - bazs[-1] + bazs[0]]
#         # Get maximum azimuthal gap
#         maxgap = max(gaps)
#         # associate with resourceID
#         origin_gaps.append([origin.resource_id.id, maxgap])

# # An exercise for users to incorporate 'gap' values into their preferred schema
# display(pd.DataFrame(origin_gaps, columns=['resource_id','gap']))

In [ ]:
# Show that the geometry data stuck
display(cat.events[0].origins[0].arrivals)

In [ ]:
# Submit the catalog back to the event bank to update
ebank.put_events(cat)

In [ ]:
# Delete `cat` and re-load to prove to ourselves that the geometry information was saved
del cat
try:
    display(cat)
except NameError:
    print('cat does not exist, as expected')

In [ ]:
# Re-load the sub-catalog
cat = ebank.get_events(event_id = _df_index.event_id)
display(cat)

In [ ]:
# View that the geometry data persist on events we modified
display(cat.events[0].origins[0].arrivals)

In [ ]:
# Load all the events and check an unmodified event
cat = ebank.get_events()


In [ ]:
# Display catalog (should see everything)
display(cat)
# Display the first event, which we did not modify
display(cat.events[0].origins[0].arrivals)